In [1]:
%load_ext cython

In [6]:
import pandas as pd

In [2]:
import sys
sys.path.append('/Users/pradap/Documents/Research/Python-Package/scaling/dmagellan/')

In [3]:
from dmagellan.core.utils import str2bytes

In [29]:
%%cython -a
#distutils: language=c++
from libcpp.string cimport string
from libcpp.map cimport map as omap
from libcpp.vector cimport vector 
from dmagellan.core.tokencontainer cimport TokenContainer
import time
from libcpp.set cimport set as oset




#########
cdef extern from "string.h":
    char *strtok_r (char *inp_str, const char *delimiters, char **) nogil
    
cdef vector[string] ctokenize_wd(const string& istring) nogil:
    cdef char* ptr1
    cdef char* pch = strtok_r(<char*> istring.c_str(), " \t\n", &ptr1)
    cdef oset[string] stokens
    cdef vector[string] otokens
    cdef string token
    while pch != NULL:
        stokens.insert(string(pch))
        pch = strtok_r(NULL, " \t\n", &ptr1)
    for token in stokens:
        otokens.push_back(token)
    return otokens
    
cdef vector[string] cremove_stopwords(vector[string]& itokens, \
            omap[string, int]& stopwords) nogil:
    cdef vector[string] otokens
    cdef string token
    for token in itokens:
        if (stopwords.find(token) == stopwords.end()):
            otokens.push_back(token)
    return otokens    

cdef string remove_stopwords(string s, omap[string, int] sw): 
    cdef vector[string] x, y
    cdef string z
    #x = ctokenize_wd(s)
    #y = cremove_stopwords(x, sw)
    #z = ' '.join(x)
    return s

cdef void convert_to_vector(string_col, vector[string]& string_vector):         
    for val in string_col:                                                      
        string_vector.push_back(val)
        
# def rem_sw(s, m):
#     x = remove_stopwords(s, m)
#     return x
def test_sw(df, attr, m):
    cdef vector[string] strings
    cdef string k, s
    convert_to_vector(df[attr], strings)                           
    st = time.time()
    for s in strings:
        k = remove_stopwords(s, m)
    print(time.time() - st)
########


cdef class QgramTokenizer:
    

In [7]:
A = pd.read_csv('../datasets/msd.csv')

In [12]:
titles = A[['title']]
titles = titles.dropna()

In [14]:
stopwords = ['a', 'an', 'and', 'are', 'as', 'at', 'be', 'by', 'for', 'from', 'has', 'he', 'in', 'is', 'it', 'its', 'on', 'that', 'the', 'to', 'was', 'were', 'will', 'with']
sw_map = dict(zip(stopwords, [0]*len(stopwords)))

In [30]:
test_sw(titles, 'title', sw_map)

23.8235681057
